<a href="https://colab.research.google.com/github/Azorios/DL1/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import ResNetForImageClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device, ' will be used.')

# transform input
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#     ]),
#     'test': transforms.Compose([
#         transforms.Resize(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#     ])
# }

batch_size = 32

# datasets
fake = load_dataset('poloclub/diffusiondb', '2m_random_10k', split='train', data_dir='./')
real = load_dataset('frgfm/imagenette', '320px', split='train+validation', data_dir='./')

fake = fake.remove_columns(['prompt', 'seed', 'step', 'cfg', 'sampler', 'width', 'height', 'user_name', 'timestamp', 'image_nsfw', 'prompt_nsfw'])
real = real.remove_columns('label')

# add label column
fake = fake.map(lambda x: {'image': x['image'], 'label':1})
real = real.map(lambda x: {'image': x['image'], 'label':0})

print(fake)
print(real)

# split fake dataset into train, validation and test sets
fake_train_testvalid = fake.train_test_split(test_size=0.2)
fake_test_valid = fake_train_testvalid['test'].train_test_split(test_size=0.5)

# split real dataset into train, validation and test sets
real_train_testvalid = real.train_test_split(test_size=0.2)
real_test_valid= real_train_testvalid['test'].train_test_split(test_size=0.5)

# combine fake and real datasets into single dataset for each split
train_dataset = concatenate_datasets([fake_train_testvalid['train'], real_train_testvalid['train']]).shuffle(seed=42)
val_dataset = concatenate_datasets([fake_test_valid['train'], real_test_valid['train']]).shuffle(seed=42)
test_dataset = concatenate_datasets([fake_test_valid['test'], real_test_valid['test']]).shuffle(seed=42)

print(train_dataset)
print(val_dataset)
print(test_dataset)

#image transforms fehlen noch => normalizen, siehe oben transforms (huggingface) wichtig!
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# freeze all parameters
for params in model.parameters(): params.requires_grad_ = False
nr_filters = model.fc.in_features  # number of input features of last layer
model.fc = nn.Linear(nr_filters, 1)